In [ ]:
import sys
# sys.path.append('../..')
import pandas as pd
import numpy as np
from estimate.irt_tune.irt_eval import fit_irt
from utils.util import irt, get_model_param
from utils.data import dataSplit_sample_basedParam
import os, warnings, random, json
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from estimate.irt_gradient_descent import calculate_ability, calculate_ability_binary, calculate_abilities_df_binary_single, calculate_abilities_df_single
from src.problem_assistant import Problem_Assiastant
from scipy.stats import spearmanr
from tqdm import tqdm
import csv

warnings.filterwarnings('ignore')

# 参数设置
max_num_ratio = 0.02
epochs = 100
rand_cnt = 1
root_path = 'data'
benchmark = 'SEEDBench_IMG'

# 获取模型信息
info_dict, info_df = get_model_param(os.path.join(root_path,'model_information_new.csv'), return_df=True)

# 加载测试模型列表
with open(os.path.join(root_path,'test_model_list.json'), 'r') as f:
    test_model_list = json.load(f)

# 准备所有模型列表
all_model_list = []
train_model_list = []
for key, sub_df in info_dict.items():
    if key == 'All':
        continue
    model_list = sub_df['models'].tolist()
    all_model_list += model_list

train_model_list = list(set(all_model_list) - set(test_model_list))


def process_benchmark(benchmark):
    print(f"\nProcessing benchmark: {benchmark}")
    
    # 原始数据路径
    orig_pt = f'model_performance/{benchmark}/{benchmark}.csv'
    df = pd.read_csv(orig_pt)
    
    # 准备数据
    train_df = df[['model_sha'] + train_model_list]
    test_df = df[['model_sha'] + test_model_list]
    full_model_list = list(train_model_list) + list(test_model_list)
    full_df = df[['model_sha'] + full_model_list]
    
    # IRT分析
    print("Running IRT analysis...")
    all_prob_df, all_model_df = irt(full_df, epochs=epochs)
    all_prob_df[['loc_diff', 'scale_diff']] = all_prob_df[['difficulty', 'difficulty_std']]
    train_prob_df, train_model_df = irt(train_df, epochs=epochs)
    train_prob_df[['loc_diff', 'scale_diff']] = train_prob_df[['difficulty', 'difficulty_std']]
    
    # 创建问题助手
    prob_ast = Problem_Assiastant(df, info_df, train_model_list, problem_difficulty=train_prob_df)
    
    # 创建文件夹结构
    data_folder = os.path.join(root_path, benchmark)
    train_folder = os.path.join(data_folder, 'train')
    test_folder = os.path.join(data_folder, 'test')
    full_folder = os.path.join(data_folder, 'full')
    
    os.makedirs(full_folder, exist_ok=True)
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)
    
    # 保存数据
    def save_data(folder, **kwargs):
        for name, data in kwargs.items():
            file_path = os.path.join(folder, f'{name}.json')
            if isinstance(data, pd.DataFrame):
                data.to_json(file_path, orient='records', lines=True)
            elif isinstance(data, (list, dict)):
                with open(file_path, 'w') as f:
                    json.dump(data, f, indent=4)
            else:
                raise ValueError(f"Unsupported data type for {name}: {type(data)}")
    
    # 保存 train 数据
    save_data(
        train_folder,
        train_model_list=train_model_list,
        train_prob_df=train_prob_df,
        train_model_df=train_model_df
    )
    
    # 保存 test 数据
    save_data(
        test_folder,
        test_model_list=test_model_list,
        all_prob_df=all_prob_df,
        all_model_df=all_model_df
    )
    
    save_data(
        full_folder,
        all_prob_df=all_prob_df,
        all_model_df=all_model_df
    )
    
    # 处理CSV到JSON转换
    file_path = os.path.join(full_folder, f'{benchmark}.csv')
    output_path = os.path.join(full_folder, f'{benchmark}.json')
    
    # 复制原始文件
    cmd = f'cp {orig_pt} {full_folder}'
    os.system(cmd)
    
    # 处理文件并保存为JSON
    def process_file_to_dict(file_path):
        result_dict = {}
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            headers = next(reader)
            model_names = headers[1:]
            
            for model_name in model_names:
                result_dict[model_name] = {}
            
            for row in reader:
                model_sha = row[0]
                for i, value in enumerate(row[1:]):
                    model_name = model_names[i]
                    result_dict[model_name][model_sha] = value # int(float(value))
        return result_dict
    
    def save_dict_to_json(result_dict, output_path):
        with open(output_path, 'w') as file:
            for model_name, model_data in result_dict.items():
                json_line = json.dumps({model_name: model_data})
                file.write(json_line + '\n')
    
    result_dict = process_file_to_dict(file_path)
    save_dict_to_json(result_dict, output_path)
    print(f"CSV to JSON conversion completed for {benchmark}")
    
    # 计算测试模型能力
    results = []
    for model_under_test in tqdm(test_model_list, desc=f"Evaluating models for {benchmark}"):
        random_data, selected_diff = prob_ast.get_random_problem([model_under_test], 1, refresh=True)
        model_result_df = calculate_abilities_df_binary_single(selected_diff, random_data)
        results.append({"model_sha": model_under_test, "ability": model_result_df['ability'].iloc[0]})
    
    with open(f'{test_folder}/test_model_df_fullPb.json', 'w') as f:
        for result in results:
            json.dump(result, f)
            f.write("\n")
    
    print(f"Completed processing for benchmark: {benchmark}")

if __name__ == "__main__":
    process_benchmark(benchmark)